In [2]:
pip install biopython lxml


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install biopython


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
from Bio import Entrez
from lxml import etree
import re
import json

def fetch_full_text(pmc_id):
    Entrez.email = "abvishwakarma@ucdavis.edu"  # Set this to your actual email address
    try:
        handle = Entrez.efetch(db='pmc', id=pmc_id, rettype='xml', retmode='xml')
        xml_data = handle.read().decode('utf-8')  # Decode bytes to string
        handle.close()
        # Check if the returned data is valid XML
        if not xml_data.strip().startswith('<'):
            raise ValueError(f"Received invalid XML for PMC ID {pmc_id}")
        return xml_data
    except Exception as e:
        print(f"Failed to fetch PMC ID {pmc_id}: {str(e)}")
        return None

In [5]:
def parse_xml(xml_data):
    if xml_data:
        try:
            root = etree.fromstring(xml_data)
            full_text = ''.join(root.xpath('.//body//text()'))  # Extracts all text within the body tag
            title = ''.join(root.xpath('//article-title//text()'))
            abstract = ''.join(root.xpath('//abstract//text()'))

            details = {
                'Title': title,
                'Abstract': abstract,
                'Animal Model Details': extract_animal_model_details(full_text),
                'Disease Model': extract_info(full_text, 'disease model'),
                'Endpoint': extract_info(full_text, 'endpoint')
            }

            return details
        except Exception as e:
            print(f"Failed to parse XML: {str(e)}")
            return None
    return None

In [6]:
def extract_animal_model_details(text):
    details = {}
    details['Age'] = extract_info(text, r'age(?:s)? of \w+\s*\w*')
    details['Weight'] = extract_info(text, r'weight[s]? of \w+\s*\w*')
    details['Sex'] = extract_info(text, r'(?:male|female)')
    return details

def extract_info(text, pattern):
    # Look for phrases that likely describe the information following the keyword
    match = re.search(rf"{pattern}[\s:]+([\w\s,]+)\b", text, re.IGNORECASE)
    return match.group(1) if match else "Not specified"

In [7]:
from Bio import Entrez
import json

def search_pubmed(diseases, keywords):
    Entrez.email = "abvishwakarma@ucdavis.edu"  # Always set this to your email address
    base_db = 'pubmed'
    results = []

    for disease in diseases:
        for keyword in keywords:
            query = f'({disease}) AND {keyword}[Title/Abstract]'# AND "animal model"[Title/Abstract]'
            handle = Entrez.esearch(db=base_db, term=query, retmax=50, usehistory='y')
            record = Entrez.read(handle)
            handle.close()
            id_list = record['IdList']

            if id_list:  # Check if there are IDs before proceeding
                webenv = record['WebEnv']
                query_key = record['QueryKey']
                fetch_handle = Entrez.efetch(db=base_db, rettype='xml', retmode='xml', webenv=webenv, query_key=query_key)
                articles = Entrez.read(fetch_handle)
                fetch_handle.close()
                results.extend(extract_pmc_ids(articles))
            else:
                print(f"No results found for query: {query}")

    return results

def extract_pmc_ids(articles):
    pmc_ids = []
    for article in articles['PubmedArticle']:
        article_ids = article['PubmedData']['ArticleIdList']
        for aid in article_ids:
            if aid.attributes.get('IdType') == 'pmc':
                pmc_ids.append(str(aid))
    return pmc_ids

In [8]:
#def main():
diseases = ['IBS', "Crohn's Disease", 'Celiac Disease']
keywords = ['preclinical', 'rat', 'rodent', 'mice', 'animal', 'pig', 'hamster']

pmc_ids = search_pubmed(diseases, keywords) # Example PMC IDs, replace with actual IDs
all_details = []

with open('pmc_results.json', 'a') as f:
        for pmc_id in pmc_ids:
            xml_data = fetch_full_text(pmc_id)
            if xml_data:
                parsed_details = parse_xml(xml_data)
                if parsed_details:
                    parsed_details['PMC ID'] = pmc_id  # Include the PMC ID in the details for reference
                    # Write each parsed detail to the JSON file
                    json.dump(parsed_details, f, indent=4)
                    f.write('\n')  # Add a newline for readability

print("Data has been written to pmc_results.json")

#if __name__ == '__main__':
 #   main()


Failed to fetch PMC ID PMC5407793: HTTP Error 400: Bad Request
Failed to fetch PMC ID PMC10994796: HTTP Error 400: Bad Request
Failed to fetch PMC ID PMC8413334: HTTP Error 400: Bad Request
Failed to fetch PMC ID PMC6767978: HTTP Error 400: Bad Request
Failed to fetch PMC ID PMC6718040: HTTP Error 400: Bad Request
Failed to fetch PMC ID PMC6594136: HTTP Error 400: Bad Request
Failed to fetch PMC ID PMC6135788: HTTP Error 400: Bad Request
Failed to fetch PMC ID PMC5564554: HTTP Error 400: Bad Request
Failed to fetch PMC ID PMC5987935: HTTP Error 400: Bad Request
Failed to fetch PMC ID PMC2785428: HTTP Error 400: Bad Request
Failed to fetch PMC ID PMC6668454: HTTP Error 400: Bad Request
Failed to fetch PMC ID PMC10186607: HTTP Error 400: Bad Request
Failed to fetch PMC ID PMC8803069: HTTP Error 400: Bad Request
Failed to fetch PMC ID PMC8017215: HTTP Error 400: Bad Request
Failed to fetch PMC ID PMC7566649: HTTP Error 400: Bad Request
Failed to fetch PMC ID PMC7069029: IncompleteRead(171

KeyboardInterrupt: 

In [8]:
# Function to extract PMC IDs from the fetched articles
def extract_pmc_ids(articles):
    pmc_ids = []
    for article in articles.get('PubmedArticle', []):
        article_ids = article['PubmedData']['ArticleIdList']
        for aid in article_ids:
            if aid.attributes.get('IdType') == 'pmc':
                pmc_ids.append(str(aid))
    return pmc_ids

# Main function to search and save PMC IDs to a JSON file
diseases = ['IBS', "Crohn's Disease", 'Celiac Disease']
keywords = ['preclinical', 'rat', 'rodent', 'mice', 'animal', 'pig', 'hamster']

pmc_ids = search_pubmed(diseases, keywords)

# Save the list of PMC IDs to a JSON file
with open('pmc_ids.json', 'w') as f:
    json.dump(pmc_ids, f, indent=4)

print("PMC IDs have been written to pmc_ids.json")


PMC IDs have been written to pmc_ids.json


In [9]:
pip install requests


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [12]:
import json
import os
from ftplib import FTP

def load_pmc_ids(filename):
    with open(filename, 'r') as f:
        pmc_ids = json.load(f)
    return pmc_ids

def download_paper(pmc_id, output_dir):
    ftp_server = 'ftp.ncbi.nlm.nih.gov'
    ftp_path = f'/pub/pmc/oa_xml/{pmc_id}.xml'

    try:
        ftp = FTP(ftp_server)
        ftp.login()
        ftp.cwd(os.path.dirname(ftp_path))
        local_filename = os.path.join(output_dir, f"{pmc_id}.xml")

        with open(local_filename, 'wb') as local_file:
            ftp.retrbinary(f'RETR {os.path.basename(ftp_path)}', local_file.write)

        print(f"Downloaded {pmc_id}.xml")
        ftp.quit()
    except Exception as e:
        print(f"Failed to download {pmc_id}.xml: {str(e)}")


pmc_ids_file = 'pmc_ids.json'  # The JSON file containing the list of PMC IDs
output_dir = 'downloaded_papers'  # Directory to save the downloaded papers

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

pmc_ids = load_pmc_ids(pmc_ids_file)

for pmc_id in pmc_ids:
    download_paper(pmc_id, output_dir)




Failed to download PMC11119472.xml: 550 /pub/pmc/oa_xml: No such file or directory
Failed to download PMC11112549.xml: 550 /pub/pmc/oa_xml: No such file or directory
Failed to download PMC11056916.xml: 550 /pub/pmc/oa_xml: No such file or directory
Failed to download PMC10769202.xml: 550 /pub/pmc/oa_xml: No such file or directory
Failed to download PMC10603947.xml: 550 /pub/pmc/oa_xml: No such file or directory
Failed to download PMC10726404.xml: 550 /pub/pmc/oa_xml: No such file or directory
Failed to download PMC10453983.xml: 550 /pub/pmc/oa_xml: No such file or directory
Failed to download PMC10318922.xml: 550 /pub/pmc/oa_xml: No such file or directory
Failed to download PMC10221257.xml: 550 /pub/pmc/oa_xml: No such file or directory
Failed to download PMC10111265.xml: 550 /pub/pmc/oa_xml: No such file or directory
Failed to download PMC9995971.xml: 550 /pub/pmc/oa_xml: No such file or directory
Failed to download PMC9977891.xml: 550 /pub/pmc/oa_xml: No such file or directory
Failed

KeyboardInterrupt: 